For this project I set out to generate fantasy point predictions in the Draft Kings scoring format in order to build a line up predictor that could be used to play in the weekly NFL DFS contests on the DraftKings site. The dataset used was downloaded from http://armchairanalysis.com/ and contained detailed play, player and game level statistics from NFL contests from 2000 - 2015 seasons.

Additionally I used draftkings salary data for 2014-2015 scraping the data from this site: 

http://rotoguru1.com/cgi-bin/fyday.pl?week=1&year=2014&game=dk&scsv=1



#### Notebook 1 - PostgreSQL Set Up
In the first notebook, I set up a SQL database to house the data. I use postgreSQL and load in the CSV files using a python script. My initial feature set was about 90K rows of player games for the past 15 years and my target was one of the columns which aggregated the draft kings fantasy score for each player for that game. My decision to use SQL instead of just using Python/pandas to query the data was due to the fact that I wanted to look at running average stats for each player for several lag periods for each game and this analysis worked much quicker in SQL then in Pandas. As such, I ended up deriving approximately 1400 features for my 90K samples by generating SQL queries using scripts in python and then running them in the PostgreSQL database to generate my dataset.
The feature set looked at running average player's, player team's and opponent team's stats as well as weather, vegas odds and player physical attributes and draft information.





#### Notebook 2 -  Feature Set Up

In the second notebook, I set up a couple functions that take the output generated by the query and format and separate it into the X , y pieces that can be fed into a typicak Scikit Learn regressor object. 
I use some standard imputer and scaling techniques from the Sklearn.preprocessing:
To scale all float and integer numeric features I use RobustScaler which scales between the 1 and 3rd quartiles to unit variance and centers on 0.
For values which are not player specific, I average across the whole sample space using an imputer
For categorical values, I create dummy variables

Lastly, in this notebook I created a function that could grab players that only player a certain number of games and/or were in a certain top fraction of average fantasy points per game. So for example, if I wanted to only look at top half of players by average fantasy points who all have played at least 3 games I could use that function to subset my data. This was meant to have a way to remove some of the outliers of for example a player who only played 2-3 games that might negatively affect the model training. 




#### Notebook 3 - Model Evaluation and Parameter Tuning

In this notebook I compared different regression algorithms looking at regularized linear regression (initially I used ElasticNet(CV) which minimizes a ordinary least square error function with a blend of L1 and L2 penalties for model complexity. Additionaly I used an Extra tree random forest ensemble as well as a Support Vector Machine regressor algorithms to genereate predictions and look at R2 scores, cross validation scores and out of sample predictions. 

I used the WR position to train and fine tune my model parameters by using a GridSearchCV set up. The random forest ensemble was providing reasonable results and speed compared to the other algorithms, however in the following notebook when I applied it to other positions I noticed that my R2 score suffered so I think I encountered an overfitting problem in this notebook and this would have to be revisited. As you will see in Notebook 4, I ended up generating the actual predictions using a linear regression model with a lasso (L1) regularization penalty. Lasso seemed like the logical choice here given that it is prone to reduce feature weights to 0 which is a desireable outcome when dealing with a large number of features as I did here. 

Lastly, in this notebook I tried a few different transformations of the response variable to see if bringing it to a more normal distribution would yield a better model fit but I did not see a significant increase in model accurace and therefore I decided to leave the response (target) variable in its original state.


#### Notebook 4 - Prediction Loop

In this notebook I predict the fantasy points for each player for each game of the 2014 and 2015 season. The loop predicts iteratively for each player by retraining each time based on all prior games. So for example, the prediction for player A for week 12 of 2015 would have a model that was trained on all data up to week 11 of 2015 which seems a closest approximation for this type of backtest. 

In a future iteration of this analysis, I intend to tune a different model for each position and do a rigorous gridsearch of hyperparameters for each, perhaps I can implement this using a cluster/pyspark to speed things up. Given that I only used my laptop to crunch the numbers and I simulated how a weekly line up predictor would operate I ended up having to have a pretty fast model to be able to go through year by year, week by week and player by player and re-train the model to predict each line. 

That is why I used Lasso to actually generate the predictions. I originally deployed the loop using random forest ensemble, which was the best performing model in the analysis in Notebook 3, however, this actually yielded a poor score when I looked at the total dataset of 2014 and 2015 for all positions so the particular parameters that the gridsearch yielded in Notebook 3 may have just been over fitted to WR position and did not perform as well for other positions.

As far as the prediction function itself, one thing to note here, is that I built in the ability to generate a sample weight to use to fit the model that would increase the weight of the player's own statistic by an input factor. So basically, the weight of each sample in the fitting would be based on the target column, which alone would increase the weight given to the higher point totals, something that is desirable since I am most interested in predicting the higher point totals anyway even if it is at the expense of some of the overall model accuracy. Secondly, since the prediction loop predicts individually by player, the prediction function allows to also increase the weight of the same player's own stats in the fitting of the model each time thus giving more weight to the players own past historic performances when fitting a unique model.

Unfortunately, the sample weighting worked without an issue when implementing a random forest but it throws an error when utilizing the canned linear regression models from Scikit learn. My guess is that it has to do with how large the feature set is but I am not 100% sure. If/when I port this to spark and set up for actual weekly use I can try coding the linear regression explicitely using weights and see if I can make it work.


#### Notebook 5 - Lineup Optimization

In this notebook I take the predictions generated for 2014 and 2015 and generate the optimal weekly line ups based on historical player salaries for each week.

This involves a memoized (result caching) type of solution to the famous knapsack problem of how to optimally fit objects with a (value, weight) attributes by maximizing value and satying under the weight constraint. In the fantasy football application, the knapsack is the line up and the weight is the salary and the value are the projected points. However, the line up optimization actually involves a lot more constraints because we need to hit an exact roster size and stay within positional max/min counts. 

An obvious choice might seem to go the way of combinatorics and just yield every single possible combination, however, the amount of combinations for a typical week grows to hundreds of billions for each particular week given that you are filling 8 roster spots and can have about 10-30 choices for each position.

To generate my optimization function I modify the code found here and employ a dynamic programming approach to optimize:

http://codereview.stackexchange.com/questions/20569/dynamic-programming-solution-to-knapsack-problem

The results are not overwhelming which was expected given that the R2 scores were still fairly low at around the 30% levels. Given the different expected point thresholds needed to win a 50/50 type contest on draftkings derived from this article: 
https://rotogrinders.com/articles/what-s-needed-to-win-on-draftkings-368296
(50/50 is a contest usually involving a large number of players (10-100) each submitting a line up with the top half being paid out as winners. Betting $50 and finishing in the top half of the field pays out $90)

Using the lower 119 point thershold the article suggests and assuming average defense perfromance (as defense is not predicted in my analysis). My model back tested to generate winning line ups for 21 of the 34 weeks in the past two years which would have yilded winnings of net $190 if you played a $50 line up every regular season week for each season. However, there are several reasons why I am very optimistic that I can improve my results as build this out for next season. 

1. Trying to train and tune separate models by position should yield better predictions.
2. I assume average defense salary and points but I can crowd source expert predictions by scraping projections from the web or make my own prediction model from this data and presumaby get better than average performance and/or better optimization.
3. One thing this back test misses is that the injury data is cruical in the nfl prediction game. So knowing detailed injury reports especially not easily quantifiable information such as beat writer reports on how players looked in practice leading up to the game and or player and coaches interview responses in the media and social media posts can yield a lot of information at uncovering optimal players and especially really undervalued ones. Because the DraftKings salaries are set at the beginning of the week, they often do not accurately reflect impact of injuries. The classic example is, salary values are set on a Monday but on Thursday a starting RB with a big salary of 8,000 from some team picks up an injury and now his talented back up with a salary of 3,000 is now going to get the start and have a great opportunity to greatly outperform on a point per salary basis. Less obviosly, injuries on the offensive line and defense also yield information that is not reflected in the pricing of the players on the sites. Because my model is largely based on a player's historical averages and somewhat on line up strength, this would not be reflected in the projection. Solving this can be approached in different ways, with the easiest being to just "hard coding" a player into the lineup and removing his salary before optimizing for all of the other spots. Identifying the players missed by the model could be done in python by scraping the web for expert predictions and identifying where a certain player's prediction in my model is vastly under or over the expert consensus. Similiarly, I could just blend the expert predictions along with my model.






If you have read so far, I would love to hear any feedback or suggestions. This has been a challenging but interesting exercise, even with seemingly clean data and a relatively small data set, a lot of the work is formating and setting the data up into the needed the format as opposed to the actual machine learning modeling part. However, now that these notebooks are built I can re run the code with different models and trying some different features from the data that I have.